# Integration

**Created**: 28 June 2022

## Environment

In [1]:
library(tidyverse)
library(data.table)

setwd("~/eQTL_pQTL_Characterization/")

source("03_Functional_Interpretation/scripts/utils/ggplot_theme.R")

── Attaching packages ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.8
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.1     ✔ forcats 0.5.1

── Conflicts ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


Attaching package: ‘data.table’


The following objects are masked from ‘package:dplyr’:

    between, first, last


The following object is masked from ‘package:purrr’:

    transpose




## Load Data

In [2]:
gene.info <- read.table("/nfs/team282/data/gains_team282/gene_info_864_20412_hla.txt")

In [3]:
head(gene.info)

,seqnames,start,end,width,strand,source,type,score,phase,gene_id,⋯,transcript_source,transcript_biotype,tag,transcript_support_level,exon_number,exon_id,exon_version,protein_id,protein_version,ccds_id
,<chr>,<int>,<int>,<int>,<chr>,<chr>,<chr>,<lgl>,<lgl>,<chr>,⋯,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>
ENSG00000230521,6,29887294,29888268,975,-,havana,gene,NA,NA,ENSG00000230521,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
ENSG00000225864,6,29722981,29723971,991,-,havana,gene,NA,NA,ENSG00000225864,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
ENSG00000227766,6,29942075,29943067,993,-,havana,gene,NA,NA,ENSG00000227766,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
ENSG00000237669,6,30006121,30007116,996,-,havana,gene,NA,NA,ENSG00000237669,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
ENSG00000271581,6,31356647,31357637,991,+,havana,gene,NA,NA,ENSG00000271581,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
ENSG00000285647,6,31367057,31376088,9032,+,havana,gene,NA,NA,ENSG00000285647,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [4]:
modules <- read.csv("~/gains_team282/nikhil/expression/gene_expression/modules.csv") %>%
    merge(., gene.info, by.x="Gene", by.y=0)

In [5]:
head(modules)

,Gene,Module,seqnames,start,end,width,strand,source,type,score,⋯,transcript_source,transcript_biotype,tag,transcript_support_level,exon_number,exon_id,exon_version,protein_id,protein_version,ccds_id
,<chr>,<chr>,<chr>,<int>,<int>,<int>,<chr>,<chr>,<chr>,<lgl>,⋯,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>
1,ENSG00000000003,Module_13,X,100627108,100639991,12884,-,ensembl_havana,gene,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
2,ENSG00000000419,Module_2,20,50934867,50958555,23689,-,ensembl_havana,gene,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
3,ENSG00000000457,Module_2,1,169849631,169894267,44637,-,ensembl_havana,gene,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
4,ENSG00000000460,Module_6,1,169662007,169854080,192074,+,ensembl_havana,gene,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
5,ENSG00000000938,Module_15,1,27612064,27635185,23122,-,ensembl_havana,gene,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
6,ENSG00000000971,Module_5,1,196652043,196747504,95462,+,ensembl_havana,gene,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [7]:
coloc.eqtl.mqtl <- read.csv("~/gains_team282/nikhil/colocalization/cis_eqtl_module_qtl.csv")

In [8]:
head(coloc.eqtl.mqtl)

,nsnps,PP.H0.abf,PP.H1.abf,PP.H2.abf,PP.H3.abf,PP.H4.abf,Gene,eQTL.Signal,Eigengene,Module.QTL,Coloc.Factor
,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<int>,<chr>,<chr>,<dbl>
1,1725,1.325296e-105,3.679035e-80,3.602294e-29,0,1,ENSG00000142089,1,ME_106_2,ME_106_2-11:1-1332794,1
2,1706,2.701629e-145,1.739698e-110,1.552930e-38,0,1,ENSG00000251661,2,ME_106_2,ME_106_2-11:1-1332794,1
3,1695,9.885529e-160,1.739698e-110,5.682324e-53,0,1,ENSG00000254910,2,ME_106_2,ME_106_2-11:1-1332794,1
4,1695,1.538896e-98,3.679035e-80,4.182879e-22,0,1,ENSG00000254910,3,ME_106_2,ME_106_2-11:1-1332794,1
5,1724,2.803508e-134,1.739698e-110,1.611491e-27,0,1,ENSG00000255328,3,ME_106_2,ME_106_2-11:1-1332794,1
6,1722,1.886565e-163,1.739698e-110,1.084421e-56,0,1,ENSG00000270030,1,ME_106_2,ME_106_2-11:1-1332794,1


In [38]:
cis.eqtl <- readRDS("~/gains_team282/eqtl/cisresults/ciseqtl_all.rds")

In [39]:
head(cis.eqtl)

snps,gene,statistic,pvalue,beta,se,chr,SNPpos,TSS,threshold
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>,<int>,<dbl>
rs114383479,ENSG00000000457,-0.06041776,0.9518546,-0.001449258,0.02398729,1,168894411,169894267,3.879445e-05
rs4656623,ENSG00000000457,-0.27193507,0.7856788,-0.003581294,0.01316967,1,168894562,169894267,3.879445e-05
rs74123028,ENSG00000000457,-0.33215439,0.7398440,-0.010030780,0.03019915,1,168894650,169894267,3.879445e-05
rs74123029,ENSG00000000457,-0.33215439,0.7398440,-0.010030780,0.03019915,1,168894789,169894267,3.879445e-05
rs74123030,ENSG00000000457,-0.33215439,0.7398440,-0.010030780,0.03019915,1,168894934,169894267,3.879445e-05
rs74123031,ENSG00000000457,-0.33215439,0.7398440,-0.010030780,0.03019915,1,168894958,169894267,3.879445e-05


## Module 101

The Chr 12 module QTL for module 101 is associated with lymphocyte and eosinophil count. It contains very fewe genes, none of which are particularly easy to interpret:
1. SUOX - sulfite oxidase - enzyme in amino acid biochemistry
2. TMEM50A - transmembrane protein 50A - tightly linked to RH haplotypes
3. RHD - Rh Blood Group D antigen
4. RPS26 - Member of the ribosomal 40S subunit

In [29]:
modules %>%
    dplyr::filter(Module == "Module_101") %>%
    dplyr::select(Gene, Module, seqnames, gene_name, gene_biotype)

Gene,Module,seqnames,gene_name,gene_biotype
<chr>,<chr>,<chr>,<chr>,<chr>
ENSG00000139531,Module_101,12,SUOX,protein_coding
ENSG00000183726,Module_101,1,TMEM50A,protein_coding
ENSG00000187010,Module_101,1,RHD,protein_coding
ENSG00000196656,Module_101,4,AC004057.1,transcribed_processed_pseudogene
ENSG00000197728,Module_101,12,RPS26,protein_coding
ENSG00000224183,Module_101,1,SDHDP6,processed_pseudogene
ENSG00000230426,Module_101,1,ERVMER61-1,lncRNA
ENSG00000250222,Module_101,5,AC008443.5,lncRNA
ENSG00000261349,Module_101,1,AL031432.2,unprocessed_pseudogene


The following genes colocalise with the module QTL on chromosome 12 and are a part of the module:
1. SUOX
2. RPS26

The following colocalises but is not in the module:
1. GDF11 - TGF beta ligand that activates SMAD family

In [32]:
coloc.eqtl.mqtl %>%
    dplyr::filter(grepl("ME_101_[0-9]-12", Module.QTL)) %>%
    dplyr::count(Gene) %>%
    merge(., gene.info, by.x="Gene", by.y="gene_id") %>%
    dplyr::select(Gene, gene_name)

Gene,gene_name
<chr>,<chr>
ENSG00000135414,GDF11
ENSG00000139531,SUOX
ENSG00000197728,RPS26


Take ME_101_1 as an example. Contains 1 CS with 6 SNPs.
1. rs1131017 - The SNP with the highest PIP is a 5' UTR variant in RPS26.
2. rs10876864 - The SNP with the second highest PIP is in an accessible region near IKZF4. However, IKZF4 has a cis-eqtl that did not colocalise.

In [33]:
susie <- fread("~/gains_team282/nikhil/colocalization/mqtl/fine_mapping/SuSiE/module_qtl_credible_sets.tsv")

In [36]:
susie %>%
    dplyr::filter(grepl("ME_101_1-12", Gene))

Gene,SNP,SNP_Prob,Credible_Set,Notes
<chr>,<chr>,<dbl>,<int>,<chr>
ME_101_1-12:54985643-57100037,rs1131017,0.61704516,1,NA
ME_101_1-12:54985643-57100037,rs10876864,0.24300396,1,NA
ME_101_1-12:54985643-57100037,rs705700,0.02947028,1,NA
ME_101_1-12:54985643-57100037,rs773114,0.02720337,1,NA
ME_101_1-12:54985643-57100037,rs1873914,0.02720337,1,NA
ME_101_1-12:54985643-57100037,rs705699,0.02453304,1,NA


In [37]:
"ENSG00000123411" %in% rownames(gene.info)

[1] TRUE

In [40]:
"ENSG00000123411" %in% cis.eqtl$gene

[1] TRUE